In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/깃허브에서받아옴')  # eda_nlp 경로를 추가
%cd /content/drive/MyDrive/깃허브에서받아옴
%pwd

Mounted at /content/drive
/content/drive/MyDrive/project_klue


'/content/drive/MyDrive/project_klue'

In [ ]:
# https://dacon.io/en/competitions/official/236037/overview/description

import pandas as pd
import numpy as np
import torch
import os
import random
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
from tqdm import tqdm
# from eda import *

# for graphing
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train_original = pd.read_csv('/content/train.csv')
train_original.drop(columns=['ID'], inplace=True)
test = pd.read_csv('/content/test.csv')
test.drop(columns=['ID'], inplace=True)
# submission = pd.read_csv('/content/sample_submission.csv')
train_original.shape, test.shape

((16541, 6), (7090, 1))

In [ ]:
import random
import pickle
import re

wordnet = {}
with open("wordnet.pickle", "rb") as f:
	wordnet = pickle.load(f)


# 한글만 남기고 나머지는 삭제
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)

	return parseText



########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################
def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break

	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")

	else:
		new_words = ""

	return new_words


def get_synonyms(word):
	synomyms = []

	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass

	return synomyms

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################
def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0

	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words

	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)

	return new_words


def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""

		if counter >= 10:
			return

	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)



def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word != ""]
	num_words = len(words)

	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1

	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))

	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))

	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))

	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))

	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))

	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)

	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	augmented_sentences.append(sentence)

	return augmented_sentences

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

CFG = {
    'EPOCHS':20,
    'LEARNING_RATE':1e-5,
    'BATCH_SIZE':32,
    'SEED':41
}

seed_everything(CFG['SEED']) # Seed 고정
device = torch.device('cuda')


def augment_data(df, text_column, label_column, num_aug=5):
    augmented_data = []
    for _, row in df.iterrows():
        augmented_sentences = EDA(row[text_column], num_aug=num_aug)
        # print('증강',augmented_sentences)
        for sentence in augmented_sentences:
            # print(sentence)
            new_row = {text_column: sentence}  # Initialize new_row with the text_column and sentence
            for label in label_column:  # Iterate through label_columns
                new_row[label] = row[label]  # Add each label column to new_row

            augmented_data.append(new_row)         # Append the new row to the list
    return pd.DataFrame(augmented_data)

In [ ]:
text_column = '문장'
label_column = ['유형','극성', '시제', '확실성','label']
augmented_data = augment_data(train_original, text_column, label_column)
augmented_data.shape

(99246, 6)

In [ ]:
augmented_data.head()

,문장,유형,극성,시제,확실성,label
0,0.75%포인트 금리 인상은 1994년 28년 이후 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
1,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
2,인상은 1994년 이후 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
3,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
4,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실


In [ ]:
train, val, train_target, val_target = train_test_split(augmented_data.drop('label', axis=1), augmented_data['label'], test_size=0.2, random_state=CFG['SEED'])
train = train.reset_index(drop=True)
val = val.reset_index(drop=True)
train_target = train_target.reset_index(drop=True)
val_target = val_target.reset_index(drop=True)
train.shape, val.shape, train_target.shape, val_target.shape

((79396, 5), (19850, 5), (79396,), (19850,))

In [ ]:
train

,문장,유형,극성,시제,확실성
0,당시 청소년의 바 보장하기 위해 게임 정하는 심야시간에 청소년에 대한 대통령령이 제...,사실형,긍정,과거,확실
1,낙화생은 옹크려져 누에 같으면서 몸뚱이가 형체가 있다.,사실형,긍정,현재,확실
2,휴일이면 자동차를 자가 정비하는 모습을 동네 곳곳에서 쉽게 볼 수 있다.,추론형,긍정,현재,확실
3,백병전 대신 지형에 매복해 활을 쏘는 게릴라전에서는 승전 소식이 들리기 시작했다.,사실형,긍정,과거,확실
4,횡령 추정액 중 2017∼2019년 사이 사라진 3억4000만원은 지난달 숨진 경리...,사실형,긍정,과거,확실
...,...,...,...,...,...
79391,한국 정부도 노인 일자리를 노 만들기 위해 노력해왔다.,사실형,긍정,과거,확실
79392,그런데 이렇게 맛 좋은 우곡주를 맛 제가 잘 팔지 못했어요.,대화형,부정,과거,확실
79393,셀트리온이 밝힌 공식적인 블록딜 사유는 ＇투자 기간별 포트폴리오 조정＇이었다.,사실형,긍정,과거,확실
79394,화장실은 단지 배 속 평안을 간구하는 생리학적 배 공간이 아니라 인류의 평안을 추구...,추론형,긍정,현재,확실


In [ ]:
model_nm = 'klue/roberta-small'
base_model = AutoModel.from_pretrained(model_nm)
tokenizer = AutoTokenizer.from_pretrained(model_nm)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
# tokenizer_len = [len(tokenizer(s)['input_ids']) for s in train['문장']]
# sns.histplot(tokenizer_len)
# plt.show()

# print(f'log value : {np.mean(tokenizer_len)+3*np.std(tokenizer_len)}')

In [ ]:
# tokenizer_log = np.log(tokenizer_len)
# sns.histplot(tokenizer_log)
# plt.show()

# print(f'log value : {np.mean(tokenizer_log)+3*np.std(tokenizer_log)}')
# print(f'original value : {np.exp(np.mean(tokenizer_log)+3*np.std(tokenizer_log))}')

In [ ]:
class SentenceTypeDataset(Dataset):
    def __init__(self, dataframe, tokenizer, labels=None):
        texts = dataframe['문장'].values.tolist()
        # print('texts', texts)
        self.texts = [tokenizer(text, padding='max_length', max_length=90, truncation=True, return_tensors='pt') for text in texts]
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        # print('text********************\n', text)
        if self.labels is not None:
            type_tmp = self.labels['type'][idx]
            polarity_tmp = self.labels['polarity'][idx]
            tense_tmp = self.labels['tense'][idx]
            certainty_tmp = self.labels['certainty'][idx]
            return text, torch.Tensor(type_tmp), torch.Tensor(polarity_tmp), torch.Tensor(tense_tmp), torch.Tensor(certainty_tmp)
        else:
            return text, torch.Tensor([-1,-1,-1,-1]), torch.Tensor([-1,-1,-1]), torch.Tensor([-1,-1,-1]), torch.Tensor([-1,-1])


In [ ]:
class SentenceClassifier(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.klue = base_model # from transformers package

        self.fc1 = nn.Linear(self.klue.config.hidden_size, 32)
        self.layernorm = nn.LayerNorm(32)
        self.dropout = nn.Dropout(p = 0.1)
        self.relu = nn.ReLU()
        self.type_clf = nn.Linear(32,4)
        self.polarity_clf = nn.Linear(32,3)
        self.tense_clf = nn.Linear(32,3)
        self.certainty_clf = nn.Linear(32,2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        # input_ids : token's id / attention_mask : make a model to focus on which token
        klue_out = self.klue(input_ids= input_ids, attention_mask = attention_mask)[0][:,0]

        x = self.fc1(klue_out)
        # print('x*************************', x)
        x = self.layernorm(x)
        x = self.dropout(x)
        x = self.relu(x)

        type_output = self.type_clf(x)
        type_output = self.softmax(type_output)
        polarity_output = self.polarity_clf(x)
        polarity_output = self.softmax(polarity_output)
        tense_output = self.tense_clf(x)
        tense_output = self.softmax(tense_output)
        certainty_output = self.certainty_clf(x)
        certainty_output = self.softmax(certainty_output)

        return type_output, polarity_output, tense_output, certainty_output

In [ ]:
from sklearn.metrics import f1_score
import numpy as np
import torch

def sentence_train(model, train_dataloader, val_dataloader, learning_rate, epochs, model_nm):
    best_val_loss = 99999999999999  # setting max (act as infinity)
    early_stopping_threshold_count = 0

    criterion = {
        'type': nn.CrossEntropyLoss().to(device),
        'polarity': nn.CrossEntropyLoss().to(device),
        'tense': nn.CrossEntropyLoss().to(device),
        'certainty': nn.CrossEntropyLoss().to(device)
    }

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    model = model.to(device)

    for epoch in range(epochs):
        total_f1_train = 0  # F1 score for training set
        total_loss_train = 0

        model.train()  # sets into the training mode

        all_true_labels = []
        all_pred_labels = []

        for train_input, type_label, polarity_label, tense_label, certainty_label in tqdm(train_dataloader):
            attention_mask = train_input['attention_mask'].to(device)
            input_ids = train_input['input_ids'].squeeze(1).to(device)
            type_label = type_label.to(device)
            polarity_label = polarity_label.to(device)
            tense_label = tense_label.to(device)
            certainty_label = certainty_label.to(device)
            optimizer.zero_grad()  # 가중치를 업데이트할 때 사용할 기울기를 초기화

            type_output, polarity_output, tense_output, certainty_output = model(input_ids, attention_mask)  # from the forward function
            # print('output***********************************************', type_output, polarity_output, tense_output, certainty_output)
            loss = 0.25 * criterion['type'](type_output, type_label) + \
                   0.25 * criterion['polarity'](polarity_output, polarity_label) + \
                   0.25 * criterion['tense'](tense_output, tense_label) + \
                   0.25 * criterion['certainty'](certainty_output, certainty_label)
            total_loss_train += loss.item()

            loss.backward()
            optimizer.step()

            # Collect true labels and predicted labels for F1 score calculation
            all_true_labels.append(type_label.cpu().numpy())  # True labels
            all_pred_labels.append(torch.argmax(type_output, dim=1).cpu().numpy())  # Predicted labels

        # Flatten the list of true and predicted labels

        all_true_labels = np.argmax(np.concatenate(all_true_labels), axis=1) if all_true_labels[0].ndim > 1 else np.concatenate(all_true_labels)
        all_pred_labels = np.argmax(np.concatenate(all_pred_labels), axis=1) if all_pred_labels[0].ndim > 1 else np.concatenate(all_pred_labels)

        print(all_true_labels)
        print(all_pred_labels)
        # Compute F1 score for training
        total_f1_train = f1_score(all_true_labels, all_pred_labels, average='weighted')  # 'weighted' for multiclass

        with torch.no_grad():  # since we should not change gradient for validation
            total_f1_val = 0  # F1 score for validation set
            total_loss_val = 0

            model.eval()  # deactivate training

            all_true_val_labels = []
            all_pred_val_labels = []

            for val_input, vtype_label, vpolarity_label, vtense_label, vcertainty_label in tqdm(val_dataloader):
                attention_mask = val_input['attention_mask'].to(device)
                input_ids = val_input['input_ids'].squeeze(1).to(device)

                vtype_label = vtype_label.to(device)
                vpolarity_label = vpolarity_label.to(device)
                vtense_label = vtense_label.to(device)
                vcertainty_label = vcertainty_label.to(device)

                vtype_output, vpolarity_output, vtense_output, vcertainty_output = model(input_ids, attention_mask)  # from the forward function

                loss = 0.25 * criterion['type'](vtype_output, vtype_label) + \
                        0.25 * criterion['polarity'](vpolarity_output, vpolarity_label) + \
                        0.25 * criterion['tense'](vtense_output, vtense_label) + \
                        0.25 * criterion['certainty'](vcertainty_output, vcertainty_label)

                total_loss_val += loss.item()

                # Collect true labels and predicted labels for F1 score calculation
                all_true_val_labels.append(vtype_label.cpu().numpy())  # True labels
                all_pred_val_labels.append(torch.argmax(vtype_output, dim=1).cpu().numpy())  # Predicted labels

            # Flatten the list of true and predicted labels
            all_true_val_labels = np.argmax(np.concatenate(all_true_val_labels), axis=1) if all_true_val_labels[0].ndim > 1 else np.concatenate(all_true_val_labels)
            all_pred_val_labels = np.argmax(np.concatenate(all_pred_val_labels), axis=1) if all_pred_val_labels[0].ndim > 1 else np.concatenate(all_pred_val_labels)

            print(all_true_val_labels)
            print(all_pred_val_labels)
            # Compute F1 score for validation
            total_f1_val = f1_score(all_true_val_labels, all_pred_val_labels, average='weighted')  # 'weighted' for multiclass

            os.makedirs("model1", exist_ok=True)

            print(f'Epochs: {epoch + 1} '
                  f'| Train Loss: {total_loss_train / len(train_dataloader): .3f} '
                  f'| Train F1 Score: {total_f1_train: .3f} '
                  f'| Val Loss: {total_loss_val / len(val_dataloader): .3f} '
                  f'| Val F1 Score: {total_f1_val: .3f}')

            if best_val_loss > total_loss_val:
                best_val_loss = total_loss_val  # saving only the best one
                torch.save(model, f"model1/{model_nm}.pt")
                print("Saved model")
                early_stopping_threshold_count = 0
            else:
                early_stopping_threshold_count += 1  # checking how many epochs have passed that val_loss didn't increase

            if early_stopping_threshold_count >= 3:  # ==> patience=1
                print("Early stopping")
                break


In [ ]:
if __name__ == '__main__':

    print('before shape', train.shape)
    train_tmp = train[['문장', '유형', '극성', '시제', '확실성']]
    train_tmp = pd.get_dummies(train_tmp, columns=['유형', '극성', '시제', '확실성'])
    print('train_tmp', train_tmp.head())

    print('after shape', train_tmp.shape)

before shape (79396, 5)
train_tmp                                                   문장  유형_대화형  유형_사실형  유형_예측형  \
0  당시 청소년의 바 보장하기 위해 게임 정하는 심야시간에 청소년에 대한 대통령령이 제...   False    True   False   
1                     낙화생은 옹크려져 누에 같으면서 몸뚱이가 형체가 있다.   False    True   False   
2           휴일이면 자동차를 자가 정비하는 모습을 동네 곳곳에서 쉽게 볼 수 있다.   False   False   False   
3      백병전 대신 지형에 매복해 활을 쏘는 게릴라전에서는 승전 소식이 들리기 시작했다.   False    True   False   
4  횡령 추정액 중 2017∼2019년 사이 사라진 3억4000만원은 지난달 숨진 경리...   False    True   False   

   유형_추론형  극성_긍정  극성_미정  극성_부정  시제_과거  시제_미래  시제_현재  확실성_불확실  확실성_확실  
0   False   True  False  False   True  False  False    False    True  
1   False   True  False  False  False  False   True    False    True  
2    True   True  False  False  False  False   True    False    True  
3   False   True  False  False   True  False  False    False    True  
4   False   True  False  False   True  False  False    False    True  
after shape (79396, 13)


In [ ]:
if __name__ == '__main__':

    train_type = train_tmp.iloc[:,1:5].values.tolist()
    train_polarity = train_tmp.iloc[:,5:8].values.tolist()
    train_tense = train_tmp.iloc[:,8:11].values.tolist()
    train_certainty = train_tmp.iloc[:,11:13].values.tolist()

    train_labels = {
        'type': train_type,
        'polarity': train_polarity,
        'tense': train_tense,
        'certainty': train_certainty
    }
    pd.DataFrame(train_labels)

In [ ]:
if __name__ == '__main__':

    val_tmp = val[['문장', '유형', '극성', '시제', '확실성']]
    val_tmp = pd.get_dummies(val_tmp, columns=['유형', '극성', '시제', '확실성'])
    # print('val_tmp', val_tmp.head())
    val_type = val_tmp.iloc[:,1:5].values.tolist()
    val_polarity = val_tmp.iloc[:,5:8].values.tolist()
    val_tense = val_tmp.iloc[:,8:11].values.tolist()
    val_certainty = val_tmp.iloc[:,11:13].values.tolist()
    val_labels = {
        'type': val_type,
        'polarity': val_polarity,
        'tense': val_tense,
        'certainty': val_certainty
    }
    pd.DataFrame(val_labels)

In [ ]:
if __name__ == '__main__':

    pd.DataFrame(train_labels).iloc[:, 1]

In [ ]:
if __name__ == '__main__':

    train_dataloader = DataLoader(SentenceTypeDataset(train_tmp, tokenizer, train_labels), batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0) # num_workers: how many subprocesses to use for data loading
    val_dataloader = DataLoader(SentenceTypeDataset(val_tmp, tokenizer, val_labels), batch_size=CFG['BATCH_SIZE'], num_workers=0)

In [ ]:
if __name__ == '__main__':

    model = SentenceClassifier(base_model)

In [ ]:
if __name__ == '__main__':

    print(len(train_dataloader.dataset))  # 데이터셋 크기 확인

    print(len(val_dataloader.dataset))    # 데이터셋 크기 확인

79396
19850


In [ ]:
if __name__ == '__main__':

    for batch in train_dataloader:
        print('batch', batch)  # 데이터가 올바르게 로드되는지 확인
        break

batch [{'input_ids': tensor([[[    0,  3625,    23,  ...,     1,     1,     1]],

        [[    0,     3, 26947,  ...,     1,     1,     1]],

        [[    0,  6910,  2259,  ...,     1,     1,     1]],

        ...,

        [[    0, 11821,  6724,  ...,     1,     1,     1]],

        [[    0, 25668,  2850,  ...,     1,     1,     1]],

        [[    0,   170,  1583,  ...,     1,     1,     1]]]), 'token_type_ids': tensor([[[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]]]), 'attention_mask': tensor([[[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        ...,

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]]])}, tensor([[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [

In [ ]:
if __name__ == '__main__':

    sentence_train(model, train_dataloader, val_dataloader, CFG['LEARNING_RATE'], CFG['EPOCHS'], 'kclue')

100%|██████████| 2482/2482 [09:45<00:00,  4.24it/s]


[1 1 1 ... 3 1 3]
[2 2 2 ... 1 1 1]


100%|██████████| 621/621 [00:46<00:00, 13.39it/s]


[1 1 1 ... 1 1 2]
[1 1 1 ... 1 1 1]
Epochs: 1 | Train Loss:  0.832 | Train F1 Score:  0.787 | Val Loss:  0.771 | Val F1 Score:  0.820
Saved model


100%|██████████| 2482/2482 [09:43<00:00,  4.25it/s]


...

[1 1 1 ... 1 1 2]
[1 1 1 ... 1 1 2]
Epochs: 19 | Train Loss:  0.559 | Train F1 Score:  0.979 | Val Loss:  0.559 | Val F1 Score:  0.977
Saved model


100%|██████████| 2482/2482 [09:42<00:00,  4.26it/s]


[1 1 1 ... 1 1 3]
[1 1 1 ... 1 1 3]


100%|██████████| 621/621 [00:46<00:00, 13.42it/s]


[1 1 1 ... 1 1 2]
[1 1 1 ... 1 1 2]
Epochs: 20 | Train Loss:  0.558 | Train F1 Score:  0.978 | Val Loss:  0.559 | Val F1 Score:  0.977
Saved model


In [ ]:
def get_type_predictions(model, loader):

    device = torch.device('cuda')
    model = model.to(device)

    type_probs, polarity_probs, tense_probs, clarity_probs = [], [], [], []
    with torch.no_grad():
        model.eval()
        for data_input, _, _, _, _ in tqdm(loader):
            attention_mask = data_input['attention_mask'].to(device)
            input_ids = data_input['input_ids'].squeeze(1).to(device)


            type_output, polarity_output, tense_output, clarity_output = model(input_ids, attention_mask)
            type_probs.append(type_output)
            polarity_probs.append(polarity_output)
            tense_probs.append(tense_output)
            clarity_probs.append(clarity_output)

    return torch.cat(type_probs).cpu().detach().numpy(), \
            torch.cat(polarity_probs).cpu().detach().numpy(), \
            torch.cat(tense_probs).cpu().detach().numpy(), \
            torch.cat(clarity_probs).cpu().detach().numpy()

In [ ]:
if __name__ == '__main__':

    model = torch.load("model1/kclue.pt")
    test_dataloader = DataLoader(SentenceTypeDataset(test, tokenizer), batch_size=CFG['BATCH_SIZE'], shuffle=False)

<ipython-input-26-c0667a2f3bc6>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("model/kclue.pt")


In [ ]:
#val_pred_type, val_pred_polarity, val_pred_tense, val_pred_certainty = get_type_predictions(model, val_dataloader)

#val_type = ['대화형' if i==0 else '사실형' if i==1 else '예측형' if i==2 else '추론형' for i in [np.argmax(p) for p in val_pred_type]]
#val_polarity = ['긍정' if i==0 else '미정' if i==1 else '부정' for i in [np.argmax(p) for p in val_pred_polarity]]
#val_type = ['과거' if i==0 else '미래' if i==1 else '현재' for i in [np.argmax(p) for p in val_pred_tense]]
#val_type = ['불확실' if i==0 else '확실' for i in [np.argmax(p) for p in val_pred_certainty]]

In [ ]:
if __name__ == '__main__':

    test_pred_type, test_pred_polarity, test_pred_tense, test_pred_certainty = get_type_predictions(model, test_dataloader)

100%|██████████| 222/222 [00:16<00:00, 13.49it/s]


In [ ]:
if __name__ == '__main__':

    test_pred_tense

In [ ]:
if __name__ == '__main__':

    sum(test_pred_tense[0])

In [ ]:
if __name__ == '__main__':

    test_type = ['대화형' if i==0 else '사실형' if i==1 else '예측형' if i==2 else '추론형' for i in [np.argmax(p) for p in test_pred_type]]
    test_polarity = ['긍정' if i==0 else '미정' if i==1 else '부정' for i in [np.argmax(p) for p in test_pred_polarity]]
    test_tense = ['과거' if i==0 else '미래' if i==1 else '현재' for i in [np.argmax(p) for p in test_pred_tense]]
    test_certainty = ['불확실' if i==0 else '확실' for i in [np.argmax(p) for p in test_pred_certainty]]

In [ ]:
if __name__ == '__main__':

    label_sum = []
    for i in range(len(test_type)):
        label_sum.append(f'{test_type[i]}-{test_polarity[i]}-{test_tense[i]}-{test_certainty[i]}')
    # os.makedirs('submission', exist_ok=True)

    # submission['label'] = label_sum
    # submission.to_csv('submission/klue1.csv', index=False)

In [ ]:
# submission